### Import stuff

In [3]:
import os
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.io import read_image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

### Custom Dataloader

In [4]:
def to_int(x):
    if (not type(x) is str and (x==None or np.isnan(x))):
        return 32;
    return ord(x)

class AsciiToDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None, height=40, width=80):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.height = height
        self.width = width
        
    def __len__(self):
        return len(self.img_labels)


    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])

        # create empty array
        arr = np.zeros([40,80], dtype="int")

        # indexes
        i, j = 0, 0

        # iterate through txt file and convert
        with open(img_path) as f:
            for line in f.readlines():
                for ch in line: 
                    if not (ch == "\n"):# leave out newline char
                        arr[j][i] = to_int(ch)
                    i += 1
                j += 1
                i = 0
        
        label = self.img_labels.iloc[idx, 1]
        return arr, label

### Make datasets and -loaders

In [5]:
annotation_file = "dataset/ascii.csv"
img_dir = "dataset/w80_h40_ascii"

test_dataset = AsciiToDataset(annotation_file, img_dir)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False) #shuffle=false to debug

train_features, train_labels = next(iter(test_dataloader))

### Define Convnet

In [6]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 4, 5) # input size, output size, kernel size
        self.pool = nn.MaxPool2d(2, 2)  # kernen size, stride
        self.pool = nn.Conv2d(4, 12, 5)
        self.fc1 = nn.Linear(12*7*17, 120)  #12*7*12 is the dimension of input at this point:
                                            # 7 = ((((40-4) /2) -4) /2) bc conv has kernel size 5
                                            # and therefore shaves off 2 left and right and maxpool
                                            # halves the size
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten to 1D
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [7]:
net = Net()

### Loss and optimizer

In [ ]:
criterion = 